In [2]:
import pandas as pd
from pandasql import sqldf
import numpy as np
from io import StringIO, BytesIO
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from cleanup.cleanup_functions_mp import create_mp_staging

pd.set_option('display.max_columns', 100)

# Chargement données

**A FAIRE** : Pour faire tourner ce notebook, ajustez le chemin ci-dessous pour pointer vers le dossier où vous stockez tous les tables fournies par la pipeline et les data-eng

In [3]:
BACK_DATA_PATH = '~/code/coralieclot/DataForGood/EclaireurPublic/13_eclaireur_public/back/data'


In [15]:
MP_PATH = os.path.join(BACK_DATA_PATH,'datasets','marches_publics','outputs','normalized_data.csv')
COMMUNITIES_PATH = os.path.join(BACK_DATA_PATH,'communities','processed_data','all_communities_data.parquet')

## ```marche_publics_normalized_data.csv```

In [5]:
%%time
mp_df= pd.read_csv(MP_PATH, sep=";",index_col=0,dtype=str)

CPU times: user 1.64 s, sys: 232 ms, total: 1.88 s
Wall time: 1.99 s


In [6]:
mp_df.shape

(476180, 21)

In [7]:
mp_df.head()

,objet,nature,datenotification,datepublicationdonnees,_type,formeprix,acheteur_nom,lieuexecution_typecode,codecpv,uid,montant,id,lieuexecution_code,acheteur_id,dureemois,procedure,lieuexecution_nom,siren,nom,type,titulaires
1221,GN9P18 - Prestation de maîtrise d'uvre pour l...,MARCHE,2020-02-07,2020-02-10,NaN,Révisable,NaN,CODE POSTAL,71335000,NaN,107400.0,20194694180000,11370,24110059300078,42,Procédure adaptée,Leucate,241100593,CA Le Grand Narbonne,CA,['TOGNELLA/CITE']
1222,20 F PDT ENT-Lot 2 : Produits d'entretien,ACCORD-CADRE,2020-01-28,2020-02-10,NaN,Ferme,NaN,CODE PAYS,39800000,NaN,52000.0,2019rDk0jbS9to00,FR,20004133300267,48,Procédure adaptée,FRANCE,200041333,CC de Parthenay-Gâtine,CC,['POLLET']
1223,20 F PDT ENT-Lot 1 : Essuyage papier et hygi�n...,ACCORD-CADRE,2020-01-28,2020-02-10,NaN,Ferme,NaN,CODE PAYS,39800000,NaN,112000.0,2019f2Gd0nDpRU00,FR,20004133300267,48,Procédure adaptée,FRANCE,200041333,CC de Parthenay-Gâtine,CC,['POLLET']
1226,"2019-08-Lot 2 Peintures, rev�tements sols souples",MARCHE,2020-01-22,2020-02-10,NaN,Ferme,NaN,CODE DEPARTEMENT,45442110,NaN,17762.4,2019sFApN1hOGo00,78,21780545600019,120,Procédure adaptée,Yvelines,217805456,Saint-Cyr-l'École,COM,['Entreprise g�n�rale de peinture A. VIGNOLA']
1227,"2019-08-Lot 1 D�molition, gros oeuvre, carrela...",MARCHE,2019-12-26,2020-02-10,NaN,Ferme,NaN,CODE DEPARTEMENT,45111100,NaN,71011.0,2019J4iAWL44B300,78,21780545600019,120,Procédure adaptée,Yvelines,217805456,Saint-Cyr-l'École,COM,['Coop�rative Moderne de Construction']


## Codes CPV

In [8]:
cpv_df = pd.read_csv("data/cpv_long.csv")
cpv_df.head()

,code,cpv_label,level
0,03,"Produits agricoles, de l'élevage, de la pêche,...",2
1,09,"Produits pétroliers, combustibles, électricité...",2
2,14,"Produits d'exploitation des mines, métaux de b...",2
3,15,"Produits alimentaires, boissons, tabac et prod...",2
4,16,Machines agricoles,2


# Nettoyage

In [9]:
mp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 476180 entries, 1221 to 1011072
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   objet                   476180 non-null  object
 1   nature                  476149 non-null  object
 2   datenotification        476180 non-null  object
 3   datepublicationdonnees  474251 non-null  object
 4   _type                   169891 non-null  object
 5   formeprix               474122 non-null  object
 6   acheteur_nom            76522 non-null   object
 7   lieuexecution_typecode  476179 non-null  object
 8   codecpv                 474432 non-null  object
 9   uid                     274773 non-null  object
 10  montant                 476180 non-null  object
 11  id                      472288 non-null  object
 12  lieuexecution_code      476174 non-null  object
 13  acheteur_id             476180 non-null  object
 14  dureemois               476180 non-nu

In [10]:
#Creating clean dataset
mp_clean = create_mp_staging(mp_df,cpv_df)

Cleaning duplicates: 476180 entries
     After removing duplicates: 473301 entries
Cleaning montant: 473301 entries
Cleaning dates: 473301
    After removing non valid dates from notification: 473234
Cleaning datenotification: 473234
    After removing non valid dates from publication: 473194
    After dropping rows before 2016: 471088
Cleaning duration of contract in months: 471088
Cleaning procedure: 471088
Share of dropped observations: 0.010693435255575645


In [11]:
mp_clean.head()

,acheteur_siren,acheteur_type,acheteur_nom,acheteur_sirene,titulaires_liste_noms,titulaires_nombre,objet,nature,_type,formeprix,lieuexecution_typecode,uid,montant,id,lieuexecution_code,dureemois,procedure,lieuexecution_nom,codecpv,cpv_8,cpv_2,cpv_2_label,cpv_8_label,obligation_publication,datenotification,datenotification_annee,datepublication,datepublication_annee,delaipublication_jours
1221,241100593,CA,CA Le Grand Narbonne,24110059300078,[TOGNELLA/CITE],1,GN9P18 - Prestation de maîtrise d'uvre pour l...,Marché,NaN,Révisable,CODE POSTAL,NaN,107400.0,20194694180000,11370,42,Procédure adaptée,Leucate,71335000,71335000,71,"Services d'architecture, services de construct...",NaN,Obligatoire,2020-02-07,2020,2020-02-10,2020,3.0
1222,200041333,CC,CC de Parthenay-Gâtine,20004133300267,[POLLET],1,20 F PDT ENT-Lot 2 : Produits d'entretien,Accord-cadre,NaN,Fermé,CODE PAYS,NaN,52000.0,2019rDk0jbS9to00,FR,48,Procédure adaptée,FRANCE,39800000,39800000,39,"Meubles (y compris les meubles de bureau), amé...",NaN,Obligatoire,2020-01-28,2020,2020-02-10,2020,13.0
1223,200041333,CC,CC de Parthenay-Gâtine,20004133300267,[POLLET],1,20 F PDT ENT-Lot 1 : Essuyage papier et hygi�n...,Accord-cadre,NaN,Fermé,CODE PAYS,NaN,112000.0,2019f2Gd0nDpRU00,FR,48,Procédure adaptée,FRANCE,39800000,39800000,39,"Meubles (y compris les meubles de bureau), amé...",NaN,Obligatoire,2020-01-28,2020,2020-02-10,2020,13.0
1226,217805456,COM,Saint-Cyr-l'École,21780545600019,[Entreprise g�n�rale de peinture A. VIGNOLA],1,"2019-08-Lot 2 Peintures, rev�tements sols souples",Marché,NaN,Fermé,CODE DEPARTEMENT,NaN,17762.0,2019sFApN1hOGo00,78,120,Procédure adaptée,Yvelines,45442110,45442110,45,Travaux de construction,Travaux de peinture de bâtiments,Optionnel,2020-01-22,2020,2020-02-10,2020,19.0
1227,217805456,COM,Saint-Cyr-l'École,21780545600019,[Coop�rative Moderne de Construction],1,"2019-08-Lot 1 D�molition, gros oeuvre, carrela...",Marché,NaN,Fermé,CODE DEPARTEMENT,NaN,71011.0,2019J4iAWL44B300,78,120,Procédure adaptée,Yvelines,45111100,45111100,45,Travaux de construction,Travaux de démolition,Obligatoire,2019-12-26,2019,2020-02-10,2020,46.0


In [83]:
mp_clean.to_csv('data/mp_staging.csv')

In [12]:
mp_clean.columns

Index(['acheteur_siren', 'acheteur_type', 'acheteur_nom', 'acheteur_sirene',
       'titulaires_liste_noms', 'titulaires_nombre', 'objet', 'nature',
       '_type', 'formeprix', 'lieuexecution_typecode', 'uid', 'montant', 'id',
       'lieuexecution_code', 'dureemois', 'procedure', 'lieuexecution_nom',
       'codecpv', 'cpv_8', 'cpv_2', 'cpv_2_label', 'cpv_8_label',
       'obligation_publication', 'datenotification', 'datenotification_annee',
       'datepublication', 'datepublication_annee', 'delaipublication_jours'],
      dtype='object')

**ATTENTION** : A confirmer si c'est ok de supprimer les lignes dupliquées

# Code pour l'étude d'une ville - Lyon

Etape 1 : on sélectionne la ville via selected_communities

In [16]:
communities_df = pd.read_parquet(path=COMMUNITIES_PATH)
communities_df = communities_df.astype({'siren':str})

NOM = 'Lyon'
TYPE = 'COM'

specific_community_df = communities_df.loc[(communities_df['nom'] == NOM) & (communities_df['type']==TYPE)]

assert len(specific_community_df)==1

community_info = specific_community_df.to_dict('records')[0]

#Also want to look up specific info about community

ELECTED_OFFICIALS_PATH = os.path.join(BACK_DATA_PATH, 'elected_officials', 'elected_officials.parquet')
mayors_df = pd.read_parquet(ELECTED_OFFICIALS_PATH).query("mandat == 'Maires'")
community_mayor = mayors_df[mayors_df['code_commune'] == community_info.get('cog')].to_dict('records')[0]

In [17]:
communities_df.shape

(36352, 17)

In [18]:

community_info

{'nom': 'Lyon',
 'siren': '216901231',
 'type': 'COM',
 'cog': '69123',
 'cog_3digits': None,
 'code_departement': '69',
 'code_departement_3digits': '069',
 'code_region': '84',
 'population': 522679,
 'epci': '200046977',
 'url_ptf': 'https://data.grandlyon.com/recherche?responsibleParty.organisationName=Ville%20de%20Lyon',
 'url_datagouv': None,
 'id_datagouv': None,
 'merge': 'ptf',
 'ptf': 'Métropole de Lyon',
 'trancheEffectifsUniteLegale': 53.0,
 'EffectifsSup50': True}

In [42]:
community_info['mayor'] = community_mayor['prenom'] + " " + community_mayor['nom']

In [43]:
community_info

{'nom': 'Lyon',
 'siren': '216901231',
 'type': 'COM',
 'cog': '69123',
 'cog_3digits': None,
 'code_departement': '69',
 'code_departement_3digits': '069',
 'code_region': '84',
 'population': 522679,
 'epci': '200046977',
 'url_ptf': 'https://data.grandlyon.com/recherche?responsibleParty.organisationName=Ville%20de%20Lyon',
 'url_datagouv': None,
 'id_datagouv': None,
 'merge': 'ptf',
 'ptf': 'Métropole de Lyon',
 'trancheeffectifsunitelegale': 53.0,
 'effectifssup50': True,
 'longitude': '4.835',
 'latitude': '45.758',
 'mayor': 'Grégory DOUCET'}

In [44]:
community_siren = community_info.get('siren')

mp_community = mp_clean[mp_clean['acheteur_siren'] == community_siren]


In [84]:
mp_community.to_csv('data/mp_staging_lyon.csv')

## Etendue de la donnée

In [17]:
mp_community[['notification_datetime','montant', 'dureemois']].describe()

,notification_datetime,montant,dureemois
count,3213,3.213000e+03,3213.000000
mean,2021-07-31 18:13:33.445378304,6.226238e+05,27.130096
min,2018-03-21 00:00:00,8.032000e+03,1.000000
25%,2020-05-19 00:00:00,6.649000e+04,12.000000
50%,2021-10-06 00:00:00,1.550000e+05,19.000000
75%,2022-10-31 00:00:00,4.400000e+05,48.000000
max,2023-12-27 00:00:00,5.500000e+07,855.000000
std,NaN,2.063722e+06,24.321961


# Montant notifié par année

In [52]:
mp_community.head()

,acheteur_siren,acheteur_type,acheteur_nom,acheteur_sirene,titulaires_liste_noms,titulaires_nombre,objet,nature,_type,formeprix,lieuexecution_typecode,uid,montant,id,lieuexecution_code,dureemois,procedure,lieuexecution_nom,codecpv,cpv_8,cpv_2,cpv_2_label,cpv_8_label,obligation_publication,datenotification,datenotification_annee,datepublication,datepublication_annee,delaipublication_jours
291948,216901231,COM,Lyon,21690123100011,"[[DESAMIANTAGE FRANCE DEMOLITION-], [CARDEM-]]",2,Groupe scolaire Kennedy ? désamiantage déconst...,Accord-cadre,NaN,Révisable,Code commune,216901231000112023O448400,440742.0,2023O448400,69123,9,Appel d'offres ouvert,A Lyon,45262660-5,45262660,45,Travaux de construction,Travaux de désamiantage,Obligatoire,2023-01-12,2023,2023-06-14,2023,153.0
292168,216901231,COM,Lyon,21690123100011,[IVECO LVI-],1,Acquisition d?un camion polybenne,Marché subséquent,NaN,Fermé,Code commune,216901231000112019K1869U00,117150.0,2019K1869U00,69123,12,Appel d'offres ouvert,A Lyon,34140000-0,34140000,34,Équipement de transport et produits auxiliaire...,NaN,Obligatoire,2019-08-14,2019,2019-11-19,2019,97.0
293247,216901231,COM,Lyon,21690123100011,[AIR CHARTER SERVICE],1,Transport musiciens tournée Russie ONL février...,Accord-cadre,NaN,Fermé,Code commune,216901231000112020L240100,180000.0,2020L240100,69123,6,Marché public négocié sans publicité ni mise e...,Auditorium / ONL,60400000-2,60400000,60,Services de transport (à l'exclusion du transp...,NaN,Obligatoire,2020-02-10,2020,2020-02-11,2020,1.0
293248,216901231,COM,Lyon,21690123100011,[ADG ENERGY-],1,Rénovation de l?installation d?éclairage entre...,Accord-cadre,NaN,Mixte,Code commune,216901231000112019K119402,269934.0,2019K119402,69123,73,Procédure adaptée,A Lyon 69004,45316000-5,45316000,45,Travaux de construction,NaN,Obligatoire,2019-04-16,2019,2020-02-11,2020,301.0
293249,216901231,COM,Lyon,21690123100011,[MMCI ART LOGISTICS],1,Transport matériel instruments ONL tournée Rus...,Accord-cadre,NaN,Fermé,Code commune,216901231000112020L237600,162340.0,2020L237600,69123,6,Marché public négocié sans publicité ni mise e...,Auditorium / ONL,60400000-2,60400000,60,Services de transport (à l'exclusion du transp...,NaN,Obligatoire,2020-02-11,2020,2020-02-11,2020,0.0


In [ ]:
def create_mp_dim_year(df):
    """Create year dimension table for staging table filtered on a specific community (WARNING : no grouping by acheteur_siren)"""

    mp_obligatoires = df[df['obligation_publication'] == 'Obligatoire']
    dim_annees_obligatoires = mp_obligatoires.groupby('datenotification_annee')[['montant']].agg({'montant':['sum', pd.Series.count]}).reset_index()
    dim_annees_obligatoires.columns = ['datenotification_annee', 'publication_obligatoire_montant', 'publication_obligatoire_nombre']

    mp_optionnels = df[df['obligation_publication'] == 'Optionnel']
    dim_annees_optionnels = mp_optionnels.groupby('datenotification_annee')[['montant']].agg({'montant':['sum', pd.Series.count]}).reset_index()
    dim_annees_optionnels.columns = ['datenotification_annee', 'publication_optionnelle_montant', 'publication_optionnelle_nombre']

    dim_annees = dim_annees_obligatoires.merge(dim_annees_optionnels, how='outer', on='datenotification_annee')

    dim_annees['montant_total'] = dim_annees['publication_obligatoire_montant']+dim_annees['publication_optionnelle_montant']
    dim_annees['tendance_montant_total']=dim_annees['montant_total'].pct_change()

    return dim_annees

In [80]:
dim_annees = create_mp_dim_year(mp_community)

In [81]:
dim_annees

,datenotification_annee,publication_obligatoire_montant,publication_obligatoire_nombre,publication_optionnelle_montant,publication_optionnelle_nombre,montant_total,tendance_montant_total
0,2018,56393489.0,143,309320.0,10,56702809.0,NaN
1,2019,238744502.0,409,1154727.0,38,239899229.0,3.230817
2,2020,289853033.0,476,1019291.0,32,290872324.0,0.212477
3,2021,505599266.0,672,2280811.0,72,507880077.0,0.746058
4,2022,402096595.0,591,3628902.0,112,405725497.0,-0.201139
5,2023,497396354.0,596,2014033.0,62,499410387.0,0.230907


In [ ]:
#NE MARCHE PLUS DEPUIS RENOMMAGE DES COLONNES - A REPRENDRE

# # Create figure
# fig = go.Figure()

# # Add bar chart (Primary Y-Axis)
# fig.add_trace(go.Bar(
#     x=data[("notification_year",)],
#     y=data[("montant",'sum')],
#     name="Montant (€)",
#     textposition="outside"
# ))

# # Add red dotted line (Secondary Y-Axis)
# fig.add_trace(go.Scatter(
#     x=data[("notification_year",)],
#     y=data[("montant",'count')],
#     name="Nombre de marchés",
#     mode="lines+markers",
#     line=dict(color="orange", dash="dot", width=2),
#     marker=dict(size=6)
# ))

# # Update layout for secondary y-axis
# fig.update_layout(
#     title="Montant total et nombre de marchés publics notifiés par année",
#     xaxis_title="Année",
#     yaxis=dict(title="Montant (€)", side="left"),
#     yaxis2=dict(
#         title="Nombre",
#         overlaying="y",  # Places second axis on top of the first
#         side="right",
#         showgrid=False,
#         color = 'orange'
#     ),
#     template="plotly_white",
#     width=700,
#     height=500,
#     legend=dict(
#         x=0.8,  # Adjust horizontal position (0 = left, 1 = right)
#         y=1.1)
# )

# # Assign second y-axis to the red dotted line
# fig.data[1].update(yaxis="y2")

# # Show plot
# fig.show()


/var/folders/kh/c8yjct5d3s95jbbrgl8221sc0000gn/T/ipykernel_34457/1613218440.py:6: PerformanceWarning:

indexing past lexsort depth may impact performance.

/var/folders/kh/c8yjct5d3s95jbbrgl8221sc0000gn/T/ipykernel_34457/1613218440.py:14: PerformanceWarning:

indexing past lexsort depth may impact performance.



# Explorations on original dataset (do not execute !)

In [ ]:
# mp_original = pd.read_json('data/decp-2019.json')
# mp_original = pd.json_normalize(mp_original['marches'])

In [ ]:
# mp_original.isna().sum()/len(mp_original)

procedure                          0.031652
modifications                      0.000000
nature                             0.006434
codeCPV                            0.003787
dureeMois                          0.000000
datePublicationDonnees             0.003617
titulaires                         0.003750
id                                 0.008239
formePrix                          0.020220
dateNotification                   0.000272
objet                              0.003080
montant                            0.000000
acheteur.id                        0.000272
source                             0.000000
lieuExecution.code                 0.003482
lieuExecution.typeCode             0.003482
lieuExecution.nom                  0.005997
lieuExecution                      1.000000
uid                                0.567934
_type                              0.683434
acheteur.nom                       0.866572
uuid                               0.860748
dateDebutExecution              

In [ ]:
# pd.json_normalize(mp_original['modifications'])

In [ ]:
# mp_original.head()

,procedure,modifications,nature,codeCPV,dureeMois,datePublicationDonnees,titulaires,id,formePrix,dateNotification,objet,montant,acheteur.id,source,lieuExecution.code,lieuExecution.typeCode,lieuExecution.nom,lieuExecution,uid,_type,acheteur.nom,uuid,dateDebutExecution,considerationsSociales,considerationsEnvironnementales,marcheInnovant,ccag,offresRecues,attributionAvance,typeGroupementOperateurs,sousTraitanceDeclaree,idAccordCadre,actesSousTraitance,modificationsActesSousTraitance,technique,modaliteExecution,TypePrix,tauxAvance,origineUE,origineFrance,created_at,typesPrix,typePrix,valeurGlobale,dateSignature,montantSubventionPublique,concessionnaires,donneesExecution,autoriteConcedante.id,autoriteConcedante.nom,term.acheteur.id,updated_at
0,Appel d'offres ouvert,[],ACCORD-CADRE,30192000,48,2020-08-06,"[{'typeIdentifiant': 'SIRET', 'id': '571722669...",20202020F1234900,Révisable,2020-08-06,"Achat et livraison de fournitures, petits maté...",4102920.24,13000548100010,data.gouv.fr_aife,22,CODE DEPARTEMENT,(22) Côtes-d'Armor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Procédure adaptée,[],MARCHE,71300000,18,2020-02-10,"[{'typeIdentifiant': 'SIRET', 'id': '394360077...",2020150980004500,Ferme et actualisable,2020-02-10,RESTRUCTURATION MA LONS LE SAUNIER PROJET ILOT...,95280.00,17210120600016,data.gouv.fr_aife,39,CODE DEPARTEMENT,(39) Jura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Appel d'offres ouvert,[],ACCORD-CADRE,90513000,36,2019-01-08,"[{'typeIdentifiant': 'SIRET', 'id': '302590898...",20192019S2228100,Ferme et actualisable,2019-01-08,"DAL-DAH2018-34 Prestation de collecte, enlèvem...",37539.00,26420030400014,data.gouv.fr_aife,42,CODE DEPARTEMENT,(42) Loire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Appel d'offres ouvert,[],ACCORD-CADRE,90513000,26,2019-01-01,"[{'typeIdentifiant': 'SIRET', 'id': '380040402...",20192019S2227900,Ferme et actualisable,2019-01-01,"DAL-DAH2018-34 Prestation de collecte, enlèvem...",1168980.82,26420030400014,data.gouv.fr_aife,42,CODE DEPARTEMENT,(42) Loire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Procédure adaptée,[],MARCHE,45252125,23,2020-02-03,"[{'typeIdentifiant': 'SIRET', 'id': '429723307...",2020150982972800,Ferme et actualisable,2020-02-03,FD RTM de la Lauzière - Torrent de la Balme - ...,42254.00,13000824600014,data.gouv.fr_aife,73,CODE DEPARTEMENT,(73) Savoie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# mp_original = mp_original[(mp_original.isna().sum()/len(mp_original) < 0.80).index.to_list()]

In [ ]:
# mp_original.to_csv('data/mp_intermediaire.csv')

In [ ]:
# mp_original = pd.read_csv('data/mp_intermediaire.csv',dtype=str,index_col=0)

In [ ]:
# mp_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1015725 entries, 0 to 1015724
Data columns (total 52 columns):
 #   Column                           Non-Null Count    Dtype 
---  ------                           --------------    ----- 
 0   procedure                        983575 non-null   object
 1   modifications                    1015725 non-null  object
 2   nature                           1009190 non-null  object
 3   codeCPV                          1011878 non-null  object
 4   dureeMois                        1015725 non-null  object
 5   datePublicationDonnees           1012028 non-null  object
 6   titulaires                       1011916 non-null  object
 7   id                               1007356 non-null  object
 8   formePrix                        995187 non-null   object
 9   dateNotification                 1015449 non-null  object
 10  objet                            1012597 non-null  object
 11  montant                          1015725 non-null  object
 12  achet

In [ ]:
# mp_original[mp_original.columns[(mp_original.isna().sum()/len(mp_original) < 0.90)]]

,procedure,modifications,nature,codeCPV,dureeMois,datePublicationDonnees,titulaires,id,formePrix,dateNotification,objet,montant,acheteur.id,source,lieuExecution.code,lieuExecution.typeCode,lieuExecution.nom,uid,_type,acheteur.nom,uuid
0,Appel d'offres ouvert,[],ACCORD-CADRE,30192000,48,2020-08-06,"[{'typeIdentifiant': 'SIRET', 'id': '571722669...",20202020F1234900,Révisable,2020-08-06,"Achat et livraison de fournitures, petits maté...",4102920.24,13000548100010,data.gouv.fr_aife,22,CODE DEPARTEMENT,(22) Côtes-d'Armor,NaN,NaN,NaN,NaN
1,Procédure adaptée,[],MARCHE,71300000,18,2020-02-10,"[{'typeIdentifiant': 'SIRET', 'id': '394360077...",2020150980004500,Ferme et actualisable,2020-02-10,RESTRUCTURATION MA LONS LE SAUNIER PROJET ILOT...,95280.0,17210120600016,data.gouv.fr_aife,39,CODE DEPARTEMENT,(39) Jura,NaN,NaN,NaN,NaN
2,Appel d'offres ouvert,[],ACCORD-CADRE,90513000,36,2019-01-08,"[{'typeIdentifiant': 'SIRET', 'id': '302590898...",20192019S2228100,Ferme et actualisable,2019-01-08,"DAL-DAH2018-34 Prestation de collecte, enlèvem...",37539.0,26420030400014,data.gouv.fr_aife,42,CODE DEPARTEMENT,(42) Loire,NaN,NaN,NaN,NaN
3,Appel d'offres ouvert,[],ACCORD-CADRE,90513000,26,2019-01-01,"[{'typeIdentifiant': 'SIRET', 'id': '380040402...",20192019S2227900,Ferme et actualisable,2019-01-01,"DAL-DAH2018-34 Prestation de collecte, enlèvem...",1168980.82,26420030400014,data.gouv.fr_aife,42,CODE DEPARTEMENT,(42) Loire,NaN,NaN,NaN,NaN
4,Procédure adaptée,[],MARCHE,45252125,23,2020-02-03,"[{'typeIdentifiant': 'SIRET', 'id': '429723307...",2020150982972800,Ferme et actualisable,2020-02-03,FD RTM de la Lauzière - Torrent de la Balme - ...,42254.0,13000824600014,data.gouv.fr_aife,73,CODE DEPARTEMENT,(73) Savoie,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015720,Procédure adaptée,"[{'objetModification': ""Avt 1 au Marché n°2020...",Accord-cadre,80500000,30,2020-07-21,"[{'typeIdentifiant': 'SIRET', 'id': '824363436...",20202020S0007501,Révisable,2020-07-21,Programme régional de formation à destination ...,0.0,20005372600028,ternum-bfc,21,Code département,(21) Côte-d'Or,NaN,Marché,NaN,69D01830-9071-479A-B35D-287429FFE7BE
1015721,Procédure adaptée,"[{'objetModification': ""Avt 1 au Marché n°2020...",Accord-cadre,80500000,30,2020-07-21,"[{'typeIdentifiant': 'SIRET', 'id': '778666453...",20202020S0009001,Révisable,2020-07-21,Programme régional de formation à destination ...,0.0,20005372600028,ternum-bfc,21,Code département,(21) Côte-d'Or,NaN,Marché,NaN,8CCCF634-832D-419E-ACAD-03058A6D4BE8
1015722,Procédure adaptée,"[{'objetModification': ""Modification n°1: exéc...",Marché,45233140,12,2019-10-07,"[{'typeIdentifiant': 'SIRET', 'id': '490921996...",20192019T0000501,Ferme,2019-10-07,La présente consultation a pour objet la rénov...,51460.0,20007053000015,ternum-bfc,71,Code département,(71) Saône-et-Loire,NaN,Marché,NaN,38294B39-5A08-43AE-BC8F-E56EDABA0739
1015723,Procédure adaptée,"[{'objetModification': ""MODIFICATION 1: Exécut...",Marché,45311200,12,2019-10-07,"[{'typeIdentifiant': 'SIRET', 'id': '389562430...",20192019T0002101,Ferme et actualisable,2019-10-07,La présente consultation a pour objet la rénov...,79600.0,20007053000015,ternum-bfc,71,Code département,(71) Saône-et-Loire,NaN,Marché,NaN,806BD81D-4E46-43D0-9E54-95418808D897


In [ ]:
# mp_df.columns

Index(['dureemois', 'nature', 'codecpv', 'acheteur_nom', 'datenotification',
       'id', 'lieuexecution_nom', 'procedure', 'montant', 'acheteur_id',
       'lieuexecution_typecode', 'uid', 'lieuexecution_code', 'formeprix',
       'datepublicationdonnees', '_type', 'objet', 'siren', 'nom', 'type',
       'titulaires'],
      dtype='object')

In [ ]:
# mp_original = mp_original[mp_original.columns[(mp_original.isna().sum()/len(mp_original) < 0.90)]]

In [ ]:
# mp_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1015725 entries, 0 to 1015724
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   procedure               983575 non-null   object
 1   modifications           1015725 non-null  object
 2   nature                  1009190 non-null  object
 3   codeCPV                 1011878 non-null  object
 4   dureeMois               1015725 non-null  object
 5   datePublicationDonnees  1012028 non-null  object
 6   titulaires              1011916 non-null  object
 7   id                      1007356 non-null  object
 8   formePrix               995187 non-null   object
 9   dateNotification        1015449 non-null  object
 10  objet                   1012597 non-null  object
 11  montant                 1015725 non-null  object
 12  acheteur.id             1014514 non-null  object
 13  source                  1015725 non-null  object
 14  lieuExecution.code     

In [ ]:
# mp_original.to_csv('data/mp_intermediaire.csv')

In [ ]:
# mp_original[mp_original['titulaires'].isna()]

,procedure,modifications,nature,codeCPV,dureeMois,datePublicationDonnees,titulaires,id,formePrix,dateNotification,objet,montant,acheteur.id,source,lieuExecution.code,lieuExecution.typeCode,lieuExecution.nom,uid,_type,acheteur.nom,uuid
420,Procédure adaptée,[],MARCHE,42961100,6,2020-02-12,NaN,2020.004100,Ferme,2020-02-11,Fourniture et installation d�un syst�me de con...,57142.7,77953712500022,data.gouv.fr_aife,38000,CODE POSTAL,GRENOBLE,NaN,NaN,NaN,NaN
579,Appel d'offres ouvert,[],ACCORD-CADRE,16700000,48,2018-11-14,NaN,20182018F1921800,Révisable,2018-11-14,"Engins porte-outils en acquisition, options, é...",3200000.0,77605646700587,data.gouv.fr_aife,02,CODE DEPARTEMENT,(02) Aisne,NaN,NaN,NaN,NaN
795,Procédure adaptée,[],MARCHE,33140000,14,2018-11-08,NaN,20182018F0026400,Ferme,2018-11-08,Acquisition de dispositifs médicaux à usage un...,150000.0,26450009100014,data.gouv.fr_aife,45,CODE DEPARTEMENT,(45) Loiret,NaN,NaN,NaN,NaN
1115,Appel d'offres restreint,[],ACCORD-CADRE,35520000,60,2018-12-21,NaN,20182018F0051900,NaN,2018-12-21,M18T20008 relatif à la fourniture d'équipement...,6666667.0,15000106300018,data.gouv.fr_aife,83,CODE DEPARTEMENT,(83) Var,NaN,NaN,NaN,NaN
1186,Appel d'offres ouvert,[],ACCORD-CADRE,50711000,48,2018-12-13,NaN,20182018T0056800,Révisable,2018-12-13,MARCHE DE MAINTENANCE ET DE TRAVAUX SUR LES IN...,0.0,17760000400044,data.gouv.fr_aife,14,CODE DEPARTEMENT,(14) Calvados,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998363,Procédure négociée ouverte,"[{'dateSignatureModification': '2022-12-30', '...",Concession de service,55510000,72,2022-08-16,NaN,2022RL02301,NaN,NaN,Concession de service public de la restauratio...,0.0,NaN,marches-publics.info,13700,Code postal,Ville de marignane,NaN,Contrat de concession,NaN,NaN
1000781,Procédure négociée ouverte,"[{'dateSignatureModification': '2020-12-08', '...",Concession de service,50118110,60,2019-07-29,NaN,2019FOURRIERE01,NaN,NaN,Concession de service public pour l'exploitati...,0.0,NaN,marches-publics.info,69000,Code postal,A LYON,NaN,Contrat de concession,NaN,6322B4D1D851F3B74BA283AD13D5DB79
1000847,Procédure négociée restreinte,"[{'dateSignatureModification': '2020-09-30', '...",Concession de service public,55241000,48,2019-10-24,NaN,2019000000006b02,NaN,NaN,Appel à candidatures pour la concession de ser...,0.0,NaN,marches-publics.info,31840,Code postal,Seilh (31),NaN,Contrat de concession,NaN,ED64EAD89BD6D1F007301EBD3CAE1253
1001173,Procédure négociée ouverte,"[{'dateSignatureModification': '2019-11-22', '...",Concession de service,85312110,72,2019-11-13,NaN,2019CC101,NaN,NaN,DELEGATION DE SERVICE PUBLIC POUR LA GESTION D...,0.0,NaN,marches-publics.info,74940,Code postal,ANNECY,NaN,Contrat de concession,NaN,C063A155D7D56D4366B88F1DC4C6FAED


In [5]:
# import json

# type_id = []
# id = []
# denomination=[]
# for titulaires in mp_original['titulaires']:
#     # a = json.loads(item)
#     # if type(item) == str:
#     #     item = item.replace("/", "")
#     #     a = eval(item)
#     #     if type(a) == list and len(a)>0:
#     #         type_id.append([titulaire.get('typeIdentifiant') for titulaire in a])
#     #         id.append([titulaire.get('id') for titulaire in a])
#     #         denomination.append([titulaire.get('denominationSociale') for titulaire in a])
#     #     else:
#     #         type_id.append(item)
#     #         id.append(item)
#     #         denomination.append(item)
#     # else :
#     #     type_id.append(item)
#     #     id.append(item)
#     #     denomination.append(item)

# pd.DataFrame({'typeid': type_id, 'id': id, 'denominationsociale': denomination})


In [ ]:
# siren_df = pd.read_parquet('../13_eclaireur_public/back/data/sirene/sirene.parquet')

In [ ]:
# siren_df.dtypes

siren                     object
is_active                   bool
raison_sociale            object
raison_sociale_prenom     object
naf8                      object
code_ju                    int64
tranche_effectif         float64
dtype: object